In [1]:
#!pip install llama-index

<br>  

# **OPTIONAL: OpenAI Ada Embeddings
***
## **WARNING: HIGHLY RECOMMEND THAT YOU RUN THIS EXERCISE LOCALLY.  
## THE EMBEDDINGS GENERATED TAKE UP 250MB OF DISK SPACE AND WILL TAKE OVER 30 MINUTES TO DOWNLOAD FROM GOOGLE COLAB

This optional section explores the process for creating text embeddings using the powerful OpenAI model `text-embedding-ada-002`.  For additional details about this process see the OpenAI [Embeddings documentation](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings). 

### Overview

Even if you don't plan on using the `text-embedding-ada-002` model for your particular use case, this exercise will, at the very least, provide you with a basis for comparison against one of the most popular (currently) embeddings frameworks on the commercial market.  While there are several ways to approach embeddings creation through the OpenAI API, we will use the code provided by LlamaIndex, as it is one the cleanest implementations that I've seen.  The output from this exercise will be a pandas DataFrame of text/vector pairs similar to what we derived from Assignment 1.2. 

In [2]:
import openai
import tiktoken
from tiktoken import Encoding
from typing import Union, List, Tuple
from getpass import getpass

In [ ]:
#set api key
openai.api_key = getpass()